<a href="https://colab.research.google.com/github/tthunga24/TAQDataAggregation/blob/main/TAQAggregation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [23]:
import pandas as pd
"""
Import NYSE TAQ data here as df using pd.read_csv()
Data should at least contain columns: BID, ASK, TIME_M, DATE, SYM_ROOT
"""

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [24]:
mktOpen = "09:30:00" # Specify market hours for use in bid/ask spread filtering
mktClose = "16:00:00"
ticker = df['SYM_ROOT'].iloc[0]

def filterZeros(df):
    df = df[(df['ASK'] != 0) & (df['BID'] != 0)]
    return df

def filterHours(df, startTime, endTime):
    df = df[pd.to_datetime(df['TIME_M'], format='%H:%M:%S.%f').dt.time >= pd.to_datetime(startTime).time()]
    df = df[pd.to_datetime(df['TIME_M'], format='%H:%M:%S.%f').dt.time <= pd.to_datetime(endTime).time()]
    return df

def filterBadSpreads(df, maxSpread, maxSpreadExt):
    time_m = pd.to_datetime(df['TIME_M'], format='%H:%M:%S.%f').dt.time
    mkt_open_time = pd.to_datetime(mktOpen).time()
    mkt_close_time = pd.to_datetime(mktClose).time()
    if (time_m <= mkt_open_time).any() or (time_m >= mkt_close_time).any():
        df = df[df['ASK'] - df['BID'] <= maxSpreadExt]
        df = df[df['ASK'] - df['BID'] >= 0]
        return df
    else:
      df = df[df['ASK'] - df['BID'] <= maxSpread]
      df = df[df['ASK'] - df['BID'] >= 0]
    return df

def calculateMidpoint(df):
    df['MIDPOINT'] = (df['ASK'] + df['BID']) / 2
    return df

def aggregateData(df, interval):
    df['TIMESTAMP'] = pd.to_datetime(df['DATE'].astype(str) + ' ' + df['TIME_M'].astype(str))
    df = df.set_index('TIMESTAMP')
    aggregated_df = df['MIDPOINT'].resample(interval).ohlc()
    return aggregated_df


In [25]:
df = df.dropna(subset=["BID", "ASK", "TIME_M"])
df = filterZeros(df)
df = filterBadSpreads(df, 0.05, 0.10) # Because extended market hours may have less liquidity, consider allowing looser spreads
df = filterHours(df, '08:00:00', "16:00:00") #24hr time format
df = calculateMidpoint(df)
df = aggregateData(df, '1min') #Specify aggregation interval
df = df.round(2) #Modify if further precision in price is required
df['SYMBOL'] = ticker
df.to_csv('data.csv')

,open,high,low,close
TIMESTAMP,,,,
2024-12-02 08:00:00,602.42,602.42,602.41,602.41
2024-12-02 08:00:01,602.41,602.42,602.41,602.42
2024-12-02 08:00:02,602.42,602.42,602.41,602.42
2024-12-02 08:00:03,602.42,602.42,602.41,602.42
2024-12-02 08:00:04,602.41,602.42,602.41,602.42
...,...,...,...,...
2024-12-02 15:59:55,603.59,603.61,603.50,603.52
2024-12-02 15:59:56,603.51,603.54,603.46,603.52
2024-12-02 15:59:57,603.52,603.58,603.50,603.54
